In [1]:
%load_ext autoreload
%autoreload 2
#%reload_ext autoreload

In [2]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import Generator
import model_definitions

### Open input file

In [53]:
SEQUENCE_LENGTH=20
ONE_CAR=True
PATH=r"D:\Users\Students\Daniele\records\OneCar_db.txt"
BATCH_SIZE=256*2
FEATURES=7 
FEATURES_PREDICTED=3
DISCARD=4

In [24]:
maxSpeed=40
frequency=0.02
maxdiff=maxSpeed*frequency
maxAngularVelocity=140 #140 degrees for second
maxAngDiff=maxAngularVelocity*frequency
print(f"Max speed diff: {maxdiff}\nMax x/z diff: {maxdiff}\nMax rot diff: {maxAngDiff}")

Max speed diff: 0.8
Max x/z diff: 0.8
Max rot diff: 2.8000000000000003


In [54]:
Generator.SEQUENCE_LENGTH=SEQUENCE_LENGTH
Generator.FEATURES=FEATURES
Generator.DISCARD=DISCARD
df=Generator.single_care_dataframe(PATH)

In [55]:
def SubtractDF(df):
    return df.groupby("RACE",group_keys=False).apply(Generator.subtraction_columns)

dfs=[]
for i in range(len(df)):
    dfs.append(SubtractDF(df[i]))
    #dfs.append(df[i])

united=dfs[0]
for i in range(1,len(df)):
    united=pd.concat([united,dfs[i] ],ignore_index=True)

In [56]:
united.tail()

,X,Z,VEL_X,VEL_Z,ROT,ANG_VEL_Y,ACC_X,ACC_Z,TILE,TILE_IND,X_RELATIVE,Z_RELATIVE,TIME,RACE,GROUP
5310813,2.3413,1.9441,-2.01551,1.05358,-5.72338,-0.186481,-5.175210,4.720117,8,16,0.375339,0.048882,0.1,167,4
5310814,2.1217,2.0481,-2.21052,0.99515,-4.38789,0.364591,0.109960,-1.190663,8,16,0.389484,0.062536,0.1,167,4
5310815,1.8849,2.1230,-2.58824,0.61740,-4.11937,-0.222150,-14.392850,-1.562214,8,16,0.402050,0.076690,0.1,167,4
5310816,1.7305,2.1145,-0.75562,-0.61419,0.49810,1.193877,19.567770,-7.124043,8,16,0.413586,0.090786,0.1,167,4
5310817,1.7298,1.9934,0.56784,-1.62790,4.00287,0.507888,22.937017,-19.221210,8,16,0.425119,0.104076,0.1,167,4


In [57]:
df_train,df_val,df_test= Generator.split_train_validation_test(united,["RACE","GROUP"])

In [58]:
df_train.head()

,X,Z,VEL_X,VEL_Z,ROT,ANG_VEL_Y,ACC_X,ACC_Z,TILE,TILE_IND,X_RELATIVE,Z_RELATIVE,TIME,RACE,GROUP
0,0.00000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,0.0,0,0
1,-0.00005,0.09370,0.000000,1.831339,0.0000,0.000000,0.000000,21.263850,4,32,-0.124001,0.390583,0.1,0,0
2,-0.00061,0.29761,-0.030470,1.845064,-0.0278,-0.024275,-1.523513,-5.043560,4,32,-0.124009,0.394551,0.1,0,0
3,-0.11072,0.43237,-0.966261,1.363851,-3.8248,-0.440160,22.816403,4.689970,4,32,-0.125485,0.400316,0.1,0,0
4,-0.09253,0.55347,0.593763,0.755228,-1.9178,0.321507,12.455210,-18.515107,4,32,-0.126719,0.407695,0.1,0,0


In [59]:
gen_train=Generator.DataGenerator(BATCH_SIZE,df_train,16000)
gen_val=Generator.DataGenerator(BATCH_SIZE,df_val,16000)
gen_test=Generator.DataGenerator(BATCH_SIZE,df_test,16000)


Length: 2655220 races: 168 n batches: 2641780 / 512
Length: 1327578 races: 168 n batches: 1314138 / 512
Length: 1328020 races: 168 n batches: 1314580 / 512


In [11]:
x_y=gen_test[0]
x=x_y[0][0]
print(x)
x_y[1][0]

tf.Tensor(
[[-2.757810e-01  1.246600e-01 -1.306900e+00  1.200000e+01  2.700000e+01
  -9.446694e-02  3.595443e-01]
 [ 3.736570e-01  2.366800e-01  9.810000e-01  1.200000e+01  2.700000e+01
  -1.044873e-01  3.896078e-01]
 [-7.494500e-02  5.423100e-01  5.647000e-01  1.200000e+01  2.700000e+01
  -1.141862e-01  4.202539e-01]
 [-3.645820e-01  4.572700e-01 -8.794000e-01  1.200000e+01  2.700000e+01
  -1.241268e-01  4.515023e-01]
 [-3.495050e-01  3.488100e-01 -8.515000e-01  1.200000e+01  2.700000e+01
  -1.347103e-01  4.831592e-01]
 [-2.982920e-01  5.352800e-01  4.440000e-01  1.600000e+01  2.800000e+01
   2.088509e-01 -4.385417e-01]
 [ 1.034670e+00  5.015900e-01  3.045200e+00  1.600000e+01  2.800000e+01
   1.883610e-01 -3.072005e-01]
 [ 2.415393e+00  3.985500e-01  5.225200e+00  1.600000e+01  2.800000e+01
   1.718392e-01 -1.733464e-01]
 [ 2.441334e+00 -4.543300e-01  4.898400e+00  1.600000e+01  2.800000e+01
   1.615462e-01 -3.984375e-02]
 [ 2.848220e-01  1.767500e-01 -5.762000e-01  1.600000e+01  2.8

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([-2.32198, -2.44539, -8.118  ])>

### Model definition

#### Train and print

In [60]:
if(ONE_CAR):
    model_definitions.cars=1
else:
    model_definitions.cars=4
model_definitions.features=FEATURES
model_definitions.sequence_length=SEQUENCE_LENGTH
model_definitions.feat_pred=FEATURES_PREDICTED

### Rules Model

In [ ]:
lstm,history=LSTM_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.01,epoch=100, dropout=0.2, cells=6 )

ev=lstm.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
lstm2,history=LSTM_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.002,epoch=100, dropout=0.02, cells=1)

ev=lstm2.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
mlp,history=MLP_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.001,epoch=100)

ev=mlp.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
siamese,history=LSTM_Siamese_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.001,epoch=100, dropout=0.02, cells=2 )

ev=siamese.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

### Physichs Model

Since physichs data can be very small, Each data is multiply by 10 

In [61]:
NAME=f"MLP_DISCARD_{DISCARD}_DELTA"
mlp_ph,history_mlp_ph=model_definitions.MLP_Train(gen_train,gen_val,learning_rate=0.0001,epoch=10,verbose=1,name=NAME,path="logs")

ev=mlp_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")
tf.saved_model.save(mlp_ph,f'saved_model/{NAME}')

Epoch 1/10
5109/5109 [==============================] - 501s 98ms/step - loss: 6.7950 - mean_absolute_error: 1.3060 - accuracy: 0.0000e+00 - val_loss: 1.3284 - val_mean_absolute_error: 0.7146 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 2/10
5109/5109 [==============================] - 499s 98ms/step - loss: 1.0709 - mean_absolute_error: 0.6130 - accuracy: 1.2743e-07 - val_loss: 1.0245 - val_mean_absolute_error: 0.5940 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 3/10
5109/5109 [==============================] - 496s 97ms/step - loss: 0.9919 - mean_absolute_error: 0.5784 - accuracy: 0.0000e+00 - val_loss: 1.0147 - val_mean_absolute_error: 0.5892 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 4/10
5109/5109 [==============================] - 495s 97ms/step - loss: 0.9888 - mean_absolute_error: 0.5769 - accuracy: 0.0000e+00 - val_loss: 1.0142 - val_mean_absolute_error: 0.5889 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 5/10
5109/5109 [==============================] - 

In [ ]:
units=32
kernel_size=1
filters=32
normalization=False
cnn_lstm,history_cnn_lstm=model_definitions.CNN_LSTM_Train(gen_train,gen_val,learning_rate=0.0001,patience=10,name=f"CNN{filters}_{kernel_size}LSTM{units}_Norm{normalization}",
                                                           path="logs_350SequenceLength",epoch=10, dropout=0.2,units=units,kernel_size=kernel_size,
                                                           filters=filters, normalization=normalization,verbose=1
                                                          )

ev=cnn_lstm.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

tf.saved_model.save(cnn_lstm,f'saved_model/CNN_LSTM{units}_{kernel_size}_{filters}_PosAssoluta')

In [ ]:
lr =1e-4
units=64
cells=3
normalization=False
name=f"LSTM{cells}_{units}_{lr}_DISCARD_{DISCARD}_DELTA"
lstm_ph,history_lstm_ph=model_definitions.LSTM_Train(gen_train,gen_val,learning_rate=lr,patience=6,name=name,path="logs",
                                                     epoch=100, dropout=0.2, cells=cells,units=units, normalization=normalization,verbose=1)

ev=lstm_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")
tf.saved_model.save(mlp_ph,f'saved_model/{name}')

Epoch 1/100
5109/5109 [==============================] - 506s 98ms/step - loss: 1.2266 - mean_absolute_error: 0.6833 - accuracy: 0.0000e+00 - val_loss: 1.0301 - val_mean_absolute_error: 0.6201 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 2/100
5109/5109 [==============================] - 501s 98ms/step - loss: 0.8623 - mean_absolute_error: 0.5619 - accuracy: 0.0000e+00 - val_loss: 0.8765 - val_mean_absolute_error: 0.5639 - val_accuracy: 2.5876e-07 - lr: 1.0000e-04
Epoch 3/100
5109/5109 [==============================] - 501s 98ms/step - loss: 0.7485 - mean_absolute_error: 0.5205 - accuracy: 0.0000e+00 - val_loss: 0.8046 - val_mean_absolute_error: 0.5356 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 4/100
5109/5109 [==============================] - 501s 98ms/step - loss: 0.6872 - mean_absolute_error: 0.4969 - accuracy: 1.2743e-07 - val_loss: 0.7680 - val_mean_absolute_error: 0.5204 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 5/100
5109/5109 [=============================

In [27]:
tf.saved_model.save(lstm_ph,f'saved_model/{name}_PosAssoluta')

INFO:tensorflow:Assets written to: saved_model/LSTM2_32_1e-05_NormFalse_PosAssoluta\assets


INFO:tensorflow:Assets written to: saved_model/LSTM2_32_1e-05_NormFalse_PosAssoluta\assets


In [19]:
model=model_definitions.CNN_LSTM_Model(0.2, units=32,filters=32,kernel_size=1, normalization=False)
model.load_weights("saved_model\CNN_LSTM32_1_32_PosAssoluta")

In [20]:
model.summary()

Model: "CNN32_1LSTM32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 30, 5)]           0         
                                                                 
 conv1d (Conv1D)             (None, 30, 32)            192       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 30, 32)           0         
 )                                                               
                                                                 
 lstm_30 (LSTM)              (None, 32)                8320      
                                                                 
 dense_6 (Dense)             (None, 3)                 99        
                                                                 
Total params: 8,611
Trainable params: 8,611
Non-trainable params: 0
___________________________________________________

In [ ]:
import tf2onnx
import onnx
input_signature=(tf.TensorSpec([None,30,5],tf.float32,name='x'))
onnx_model,_=tf2onnx.converter.from_keras(model,input_signature,opset=9)